## <font color='FFFFFF'>Overview: How to use chains with Langchain</font>
![Pokedex](https://emojis.slackmojis.com/emojis/images/1643517291/33292/pokedex.gif?1643517291 "Pokedex")

In this notebook you will find a use case of Chains with the framework Langchain. For this, i created a Pokedex: a virtual assistant using LLM models, able to help pokemon trainers to get information about pokemon and types, using the [Pokemon API](https://beta.pokeapi.co/graphql/console/) with GraphQL as the information source. This notebook is divided in 4 sections:


0.   Install dependencies
1.   Get Pokemon data from API
2.   Handle LLM calls
3.   Let's ask some stuff to the Pokedex



**NOTE**

This is a simple example of the use of Chains with langchain, applied to a virtual assistant. I simplified the solution to work with specific type of question, so i'm not considering any data validation in the process. **This notebook should not be considered a complete solution for a virtual assistant** The main intention of this notebook is to give an overview of how can use LLM applied to different tasks, and how to use Langchain Chains to concatenate tasks.


## <font color='fcc6a0'>Section 0</font>: Install dependencies

We use graphql to connect with pokemon Api, and langchain as the main framework for the development of the Pokedex. In this section we'll find:



1.   Dependencies installation
2.   Loading of the env variables
3.   Load of the LLM




In [1]:
#Dependencies Installation
!pip install httpx gql requests_toolbelt
!pip install langchain langchain_openai langchain_community langchain_core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 615.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 858.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [3]:
# Env variable to be loaded

# If you use this notebook in googlecolab, you can add OPENAI_API_KEY as secret key and use it:
# from google.colab import userdata
# OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

OPENAI_API_KEY = "<INSERT YOUR OPENAI API KEY HERE>"

In [19]:
#LLM Model Instantiation
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_openai import OpenAI

llm = OpenAI(openai_api_key=OPENAI_API_KEY,  temperature=0.5)


## <font color='FFC7BA'>Section 1</font>: Get Pokemon data from API

###1.0 Declare queries for GQL

In [9]:
#Query for GQL
pokemon_evolution_line = """
  pokemon_v2_pokemonspecy {
    evolves_from_species_id
    evolution_chain_id
    pokemon_v2_evolutionchain {
      pokemon_v2_pokemonspecies {
        name
      }
    }
  }
  """
pokemon_encounters = """
    pokemon_v2_encounters(distinct_on: location_area_id) {
      pokemon_v2_locationarea {
        name
      }
    }
  """

pokemon_abilities = """
    pokemon_v2_pokemonabilities {
      pokemon_v2_ability {
        name
      }
    }
  """

pokemon_eficacies = """
    pokemon_v2_pokemontypes {
      pokemon_v2_type {
        name
        pokemonV2TypeefficaciesByTargetTypeId(where: {damage_factor: {_gte: 200}}, order_by: {damage_factor: desc}) {
          damage_factor
          pokemon_v2_type {
            name
            pokemon_v2_pokemontypes(limit: 3) {
              pokemon_v2_pokemon {
                name
              }
            }
          }
        }
        pokemon_v2_typeefficacies (where: {damage_factor: {_gte: 200}}, order_by: {damage_factor: desc}) {
          damage_factor
          pokemonV2TypeByTargetTypeId {
            name
          }
        }
      }
    }
  """
pokemon_query_str = f"""
  query PokemonEffectiveness ($pokemon_name: String!) {{
    pokemon_v2_pokemon(where: {{name: {{_eq: $pokemon_name}}}}) {{
      id
      name
      height
      weight
      {pokemon_evolution_line}
      {pokemon_encounters}
      {pokemon_abilities}
      {pokemon_eficacies}
    }}
  }}
"""

### 1.1: get_pokemon_info

In [10]:
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport
import json

URL_POKEMON = "https://beta.pokeapi.co/graphql/v1beta"

transport = AIOHTTPTransport(url=URL_POKEMON)
client = Client(transport=transport, fetch_schema_from_transport=True)

def simplify_pokemon_data(json_data):
    pokemon = json_data['pokemon_v2_pokemon'][0]
    evolution_chain = pokemon['pokemon_v2_pokemonspecy']['pokemon_v2_evolutionchain']['pokemon_v2_pokemonspecies']
    encounter_locations = [location['pokemon_v2_locationarea']['name'] for location in pokemon['pokemon_v2_encounters']]
    abilities = [ability['pokemon_v2_ability']['name'] for ability in pokemon['pokemon_v2_pokemonabilities']]
    type_data = pokemon['pokemon_v2_pokemontypes'][0]['pokemon_v2_type']
    type_name = type_data['name']
    effective_to = type_data["pokemon_v2_typeefficacies"]
    effectiveness = {
        'not_very_effective': [t['pokemon_v2_type']['name'] for t in type_data['pokemonV2TypeefficaciesByTargetTypeId'] if t['damage_factor'] == 200],
        'super_effective': [t["pokemonV2TypeByTargetTypeId"]["name"] for t in effective_to if t['damage_factor'] == 200]
    }

    simplified_string = f"{pokemon['name'].capitalize()} (ID: {pokemon['id']})\n"
    simplified_string += f"Height: {pokemon['height']}\n"
    simplified_string += f"Weight: {pokemon['weight']}\n\n"
    simplified_string += "Evolution chain:\n"
    for i, species in enumerate(evolution_chain, 1):
        simplified_string += f"{i}. {species['name']}\n"
    simplified_string += "\nEncounter locations:\n"
    for location in encounter_locations:
        simplified_string += f"- {location}\n"
    simplified_string += "\nAbilities:\n"
    for ability in abilities:
        simplified_string += f"- {ability}\n"
    simplified_string += f"\nType: {type_name.capitalize()}\n"
    simplified_string += "- Super effective against: " + ", ".join(effectiveness['super_effective']) + "\n"
    simplified_string += "- Not very effective against: " + ", ".join(effectiveness['not_very_effective'])

    return simplified_string

async def get_pokemon_info(data):
  pokemon_data = data["pokemon_data"]
  pokemon_name = pokemon_data["pokemon"]

  params = {"pokemon_name": pokemon_name}
  pokemon_query = gql(pokemon_query_str)

  # Execute the query on the transport
  result = await client.execute_async(pokemon_query, variable_values=params)
  # result = {"pokemon_v2_pokemon": [{"id": 38, "name": "ninetales", "height": 11, "weight": 199, "pokemon_v2_pokemonspecy": {"evolves_from_species_id": 37, "evolution_chain_id": 15, "pokemon_v2_evolutionchain": {"pokemon_v2_pokemonspecies": [{"name": "vulpix"}, {"name": "ninetales"},{"name": "articuno"}]}}, "pokemon_v2_encounters": [{"pokemon_v2_locationarea": {"name": "abundant-shrine-area"}}], "pokemon_v2_pokemonabilities": [{"pokemon_v2_ability": {"name": "flash-fire"}}, {"pokemon_v2_ability": {"name": "drought"}}], "pokemon_v2_pokemontypes": [{"pokemon_v2_type": {"name": "fire", "pokemonV2TypeefficaciesByTargetTypeId": [{"damage_factor": 200, "pokemon_v2_type": {"name": "ground", "pokemon_v2_pokemontypes": [{"pokemon_v2_pokemon": {"name": "sandshrew"}}, {"pokemon_v2_pokemon": {"name": "sandslash"}}, {"pokemon_v2_pokemon": {"name": "nidoqueen"}}]}}, {"damage_factor": 200, "pokemon_v2_type": {"name": "water", "pokemon_v2_pokemontypes": [{"pokemon_v2_pokemon": {"name": "squirtle"}}, {"pokemon_v2_pokemon": {"name": "wartortle"}}, {"pokemon_v2_pokemon": {"name": "blastoise"}}]}}, {"damage_factor": 200, "pokemon_v2_type": {"name": "rock", "pokemon_v2_pokemontypes": [{"pokemon_v2_pokemon": {"name": "geodude"}}, {"pokemon_v2_pokemon": {"name": "graveler"}}, {"pokemon_v2_pokemon": {"name": "golem"}}]}}, {"damage_factor": 100, "pokemon_v2_type": {"name": "poison", "pokemon_v2_pokemontypes": [{"pokemon_v2_pokemon": {"name": "bulbasaur"}}, {"pokemon_v2_pokemon": {"name": "ivysaur"}}, {"pokemon_v2_pokemon": {"name": "venusaur"}}]}}, {"damage_factor": 100, "pokemon_v2_type": {"name": "ghost", "pokemon_v2_pokemontypes": [{"pokemon_v2_pokemon": {"name": "gastly"}}, {"pokemon_v2_pokemon": {"name": "haunter"}}, {"pokemon_v2_pokemon": {"name": "gengar"}}]}}, {"damage_factor": 100, "pokemon_v2_type": {"name": "electric", "pokemon_v2_pokemontypes": [{"pokemon_v2_pokemon": {"name": "pikachu"}}, {"pokemon_v2_pokemon": {"name": "raichu"}}, {"pokemon_v2_pokemon": {"name": "magnemite"}}]}}, {"damage_factor": 100, "pokemon_v2_type": {"name": "psychic", "pokemon_v2_pokemontypes": [{"pokemon_v2_pokemon": {"name": "abra"}}, {"pokemon_v2_pokemon": {"name": "kadabra"}}, {"pokemon_v2_pokemon": {"name": "alakazam"}}]}}, {"damage_factor": 100, "pokemon_v2_type": {"name": "dragon", "pokemon_v2_pokemontypes": [{"pokemon_v2_pokemon": {"name": "dratini"}}, {"pokemon_v2_pokemon": {"name": "dragonair"}}, {"pokemon_v2_pokemon": {"name": "dragonite"}}]}}, {"damage_factor": 100, "pokemon_v2_type": {"name": "normal", "pokemon_v2_pokemontypes": [{"pokemon_v2_pokemon": {"name": "pidgey"}}, {"pokemon_v2_pokemon": {"name": "pidgeotto"}}, {"pokemon_v2_pokemon": {"name": "pidgeot"}}]}}, {"damage_factor": 100, "pokemon_v2_type": {"name": "dark", "pokemon_v2_pokemontypes": [{"pokemon_v2_pokemon": {"name": "umbreon"}}, {"pokemon_v2_pokemon": {"name": "murkrow"}}, {"pokemon_v2_pokemon": {"name": "sneasel"}}]}}, {"damage_factor": 100, "pokemon_v2_type": {"name": "fighting", "pokemon_v2_pokemontypes": [{"pokemon_v2_pokemon": {"name": "mankey"}}, {"pokemon_v2_pokemon": {"name": "primeape"}}, {"pokemon_v2_pokemon": {"name": "poliwrath"}}]}}, {"damage_factor": 100, "pokemon_v2_type": {"name": "flying", "pokemon_v2_pokemontypes": [{"pokemon_v2_pokemon": {"name": "charizard"}}, {"pokemon_v2_pokemon": {"name": "butterfree"}}, {"pokemon_v2_pokemon": {"name": "pidgey"}}]}}], "pokemon_v2_typeefficacies": [{"damage_factor": 200, "pokemonV2TypeByTargetTypeId": {"name": "steel"}}, {"damage_factor": 200, "pokemonV2TypeByTargetTypeId": {"name": "ice"}}, {"damage_factor": 200, "pokemonV2TypeByTargetTypeId": {"name": "grass"}}, {"damage_factor": 200, "pokemonV2TypeByTargetTypeId": {"name": "bug"}}, {"damage_factor": 100, "pokemonV2TypeByTargetTypeId": {"name": "ground"}}, {"damage_factor": 100, "pokemonV2TypeByTargetTypeId": {"name": "ghost"}}, {"damage_factor": 100, "pokemonV2TypeByTargetTypeId": {"name": "electric"}}, {"damage_factor": 100, "pokemonV2TypeByTargetTypeId": {"name": "psychic"}}, {"damage_factor": 100, "pokemonV2TypeByTargetTypeId": {"name": "dark"}}, {"damage_factor": 100, "pokemonV2TypeByTargetTypeId": {"name": "normal"}}, {"damage_factor": 100, "pokemonV2TypeByTargetTypeId": {"name": "fairy"}}, {"damage_factor": 100, "pokemonV2TypeByTargetTypeId": {"name": "fighting"}}, {"damage_factor": 100, "pokemonV2TypeByTargetTypeId": {"name": "flying"}}, {"damage_factor": 100, "pokemonV2TypeByTargetTypeId": {"name": "poison"}}]}}]}]}
  return simplify_pokemon_data(result)

In [11]:
# Let's test the API:
pk_data_example = {"pokemon_data":{"pokemon":"squirtle"}}
pokemon_info_output = await get_pokemon_info(pk_data_example)
print(pokemon_info_output)

[{'damage_factor': 200, 'pokemonV2TypeByTargetTypeId': {'name': 'ground'}}, {'damage_factor': 200, 'pokemonV2TypeByTargetTypeId': {'name': 'rock'}}, {'damage_factor': 200, 'pokemonV2TypeByTargetTypeId': {'name': 'fire'}}]
['ground: 200', 'rock: 200', 'fire: 200']
Squirtle (ID: 7)
Height: 5
Weight: 90

Evolution chain:
1. squirtle
2. wartortle
3. blastoise

Encounter locations:
- vermilion-city-area
- pallet-town-area
- lumiose-city-area
- seaward-cave-area

Abilities:
- torrent
- rain-dish

Type: Water
- Super effective against: ground, rock, fire
- Not very effective against: grass, electric


### 1.2 get_type_info

In [50]:
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport

URL_POKEMON = "https://beta.pokeapi.co/graphql/v1beta"

transport = AIOHTTPTransport(url=URL_POKEMON)
client = Client(transport=transport, fetch_schema_from_transport=True)

def format_type_info(type_data):
  t = type_data["pokemon_v2_type"][0]
  weak_to = t["pokemonV2TypeefficaciesByTargetTypeId"]
  weak_to_types = [f"""{element["pokemon_v2_type"]["name"]}: {element["damage_factor"]}""" for element in weak_to]
  # For this, i added a "fakeType" as weak type, to make sure that the LLM is using the information that comes from the source, and not it's inner information
  str_weak_to = f"fakeType: 200, " + ", ".join(weak_to_types)

  effective_to = t["pokemon_v2_typeefficacies"]
  effective_types = [f"""{element["pokemonV2TypeByTargetTypeId"]["name"]}: {element["damage_factor"]}""" for element in effective_to]
  str_effective = ", ".join(effective_types)

  list_of_pokemon = t["pokemon_v2_pokemontypes"][:15]
  pokemons = [f"""{element["pokemon_v2_pokemon"]["name"]}""" for element in list_of_pokemon]
  pokemons_str = ", ".join(pokemons)
  output = f"""
    type: {t["name"]}
    {t["name"]} is effective against these types: {str_effective}
    these types are effective against {t["name"]}: {str_weak_to}
    {t["name"]}-type pokemon: {pokemons_str}
  """

  return output

async def get_type_info(data):
  pokemon_data = data["pokemon_data"]
  pokemon_type = pokemon_data["pokemon_type"]
  print(f"[get_battle_advice_by_type] pokemon_type: {pokemon_type}")

  params = {"pokemon_type": pokemon_type}

  pokemon_query = gql(
    """
      query TypeEffectiveness ($pokemon_type: String!) {
        pokemon_v2_type(where: {name: {_eq: $pokemon_type}}) {
          name
          pokemonV2TypeefficaciesByTargetTypeId(where: {damage_factor: {_gte: 200}}, order_by: {damage_factor: desc}) {
            damage_factor
            pokemon_v2_type {
              name
            }
          }
          pokemon_v2_typeefficacies(where: {damage_factor: {_gte: 200}}, order_by: {damage_factor: desc}) {
            damage_factor
            pokemonV2TypeByTargetTypeId {
              name
            }
          }
          pokemon_v2_pokemontypes {
            pokemon_v2_pokemon {
              name
            }
          }
        }
      }
  """
  )

  # Execute the query on the transport
  result = await client.execute_async(pokemon_query, variable_values=params)
  return format_type_info(result)

In [13]:
# Let's test the API:
t_data_example = {"pokemon_data":{"pokemon_type":"water"}}
type_info_output = await get_type_info(t_data_example)
print(type_info_output)

[get_battle_advice_by_type] pokemon_type: water

    type: water
    water is effective against these types: ground: 200, rock: 200, fire: 200
    these types are effective against water: fakeType: 200, grass: 200, electric: 200
    water-type pokemon: squirtle, wartortle, blastoise, psyduck, golduck, poliwag, poliwhirl, poliwrath, tentacool, tentacruel, slowpoke, slowbro, seel, dewgong, shellder
  


## <font color='FFDF84'>Section 2</font>: Handle LLM calls

We define four escenarios for this pokedex. Each one define a type of query. Not an specific query. This is important. Each category represents an universe of question that can be respond with the same information source. These categories are:


1.   **pokemon_info**: information about a Pokémon
2.   **type_info**: information about a pokemon type
3.   **smalltalks**
4.   **out_of_context**: question does not fit into any of the predefined categories

In [44]:
# Load_Prompts
prompt_classifier = """
Your task is to classify the user's question into one of these four categories and provide the appropriate label based on the instructions inside <<<>>>:

<<<
Instructions
 - Determine if the question falls into one of the following categories:
    1. **pokemon_info:** User is requesting information about a Pokémon, such as its type, weight, evolution, habitat or effectivenes
    2. **type_info:** User is requesting general information about an specific types of Pokémon.
    4. **smalltalk:** User's question is conversational and pertains to greetings, farewells, gratitude, personality questions, etc. This includes informal but specific conversations
    5. **out_of_context:** User's question does not fit into any of the predefined categories.
 - Detect (in case that exists) a pokemon entity and type in the user input
>>>

###
Examples
User Input: "What is the height of Charizard?"
Classifier Output: {{"intent": "pokemon_info", "pokemon": "charizard"}}

User Input: "Which pokemon are better against Jigglypuff?"
Classifier Output: {{"intent": "pokemon_info", "pokemon": "jigglypuff"}}

User Input: "Which pokemon are better against water-type?"
Classifier Output: {{"intent": "type_info", "pokemon_type": "water"}}

User Input: "How is the weather today?"
Classifier Output: {{"intent": "out_of_context"}}
###

{format_instructions}


User input: {user_input}
Classifier Output:
"""

prompt_pokemon_info ="""
Important: The following task requires you to strictly adhere to the provided **source** information . Do not use any external knowledge or make assumptions that are not supported by the source.

<<<
Instructions:
- Your task is to answer the user's question using only the information provided in the **source** reference.
- It is crucial that you do not include any external information or rely on prior knowledge that is not present in the **source**.
- If the answer is not found in the **source**, simply state that you do not know the answer.
>>>

###
Source
{context}
###

User's question: "{question}"
Assistant's answer:
"""
prompt_type_info = """
You will be given a user's query and your task is to answer  as comprehensively and organized as possible using the provided information in the source provided.

Example 1:
Source:
  type: water
  pokemon is effective against these types:
    rock: 200, ground: 200, fire: 200, poison: 100, bug: 100, ghost: 100
  types effective against pokemon:
    electric: 200, grass: 200, flying: 100, poison: 100, ground: 100, rock: 100, bug: 100
User Query: "What's effective against water-type Pokémon?"
Response: "Electric-type and Grass-type are the most effective against water-type Pokémon."

Example 2:
Source:
  type: water
  pokemon is effective against these types:
    rock: 200, ground: 200, fire: 200, poison: 100, bug: 100, ghost: 100
  types effective against pokemon:
    electric: 200, grass: 200, flying: 100, poison: 100, ground: 100, rock: 100, bug: 100
User Query: "I'd like to know who my water-type Pokémon is effective against."

Response: "Water-type Pokémon are effective against rock, ground, and fire Pokémon."

<<<
Instructions:
1. Read the user's query and the source carefully.
2. Answer the user's question based solely on the information in **Source**.
3. If the answer is not in the **Source**, inform the user that you don't have a response for that question.
>>>

Let's Begin

###
Source:
{context}
###
User Query: {question}
Response:
"""

prompt_smalltalk = """
You are a friendly Pokédex assistant to handle casual conversations and smalltalk. You are designed to respond in a helpful and approachable manner, providing information about Pokémon and assisting with battle strategies.

Instructions:
- Imagine you are the friendly Pokédex assistant interacting with a Pokémon Trainer.
- Respond to the user's queries in a friendly and informative way.
- Provide Pokémon-related information and offer assistance with battle strategies when appropriate.
- Emphasize the Pokédex's willingness to help and its expertise in all things Pokémon.

User Query: {question}
Response:
"""

In [55]:
# MAIN
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.globals import set_debug
from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough,
    RunnableLambda
)

# If you want to see all the call traces and logic that Langchain does in background, you can uncomment set_debug(True)
# set_debug(True)

FALLBACK_RESPONSE = "Sorry, it's seems that i'm not able to answer that. As a Pokedex, my knowledge is limited only to the Pokemon World."

class PokemonData(BaseModel):
    intent: str = Field(description="user intent: [pokemon_info, type_info, smalltalk, out_of_context]")
    pokemon: str = Field(description="pokemon detected in user input")
    pokemon_type: str = Field(description="type detected in user input")

class Intent:
  POKEMON_INFO = "pokemon_info"
  TYPE_INFO = "type_info"
  SMALLTALK = "smalltalk"
  OUT_OF_CONTEXT = "out_of_context"


async def get_chain_category():
    """get the chain to determine the correct category for the user input"""
    parser = JsonOutputParser(pydantic_object=PokemonData)
    prompt = PromptTemplate(template=prompt_classifier,
                            input_variables=["user_input"],
                            partial_variables={"format_instructions": parser.get_format_instructions()})
    chain = prompt | llm | parser
    return chain

async def get_route(data):
  pokemon_data = data["pokemon_data"]
  question = data["question"]
  intent = pokemon_data["intent"]

  output = ""
  match intent:
    case Intent.POKEMON_INFO:
        retrieval = RunnableParallel({"context": RunnableLambda(get_pokemon_info), "question": lambda x: x["question"]})
        prompt = PromptTemplate(template=prompt_pokemon_info, input_variables=["question", "context"])
        chain = retrieval | prompt | llm
        return chain
    case Intent.TYPE_INFO:
        retrieval = RunnableParallel({"context": RunnableLambda(get_type_info), "question": lambda x: x["question"]})
        prompt = PromptTemplate(template=prompt_pokemon_info, input_variables=["question", "context"])
        chain = retrieval | prompt | llm
        return chain
    case Intent.SMALLTALK:
        prompt = PromptTemplate(template=prompt_smalltalk, input_variables=["question", "context"])
        chain = prompt | llm
        return chain
    case Intent.OUT_OF_CONTEXT:
        return FALLBACK_RESPONSE

chain_category = await get_chain_category();
runnable = RunnableParallel(
    question=RunnablePassthrough(),
    pokemon_data=chain_category,
)
full_chain = runnable | RunnableLambda(get_route)


## <font color='C0F3A1'>Section 3</font>: Let's ask some stuff to the Pokedex

I defined a couple questions to each category, just to show how the assistant work in each scenario and check the process and calls to LLM that are used to be able to give an answer.

In [67]:
# Pokemon info question
query_1 = "Do you know metapod"
query_2 = "Do you know squirtle"
query_3 = "Im not sure if ponyta is water-type or plant-type"
query_4 = "vulpix evolves in onix?"
query_5 = "What does articuno evolve into?"
query_6 = "Where can i find a pikachu?"
query_7 = "Is vaporeon a fire-type?"
query_8 = "Ash has a Chansey?"

# Type info question
query_9 = "can you tell me a water-type pokemon?"
query_10 = "can you name 4 ground-type pokemon?"
query_11 = "can you tell me something about electric-type pokemon?"
query_12 = "Which pokemon is stronger against fire-type?"

# Smalltalks
query_13 = "Who are you?"
query_14 = "How can you help me?"

# Out of context
query_15 = "Can you give a recipe of a chocolate cake"


In [68]:
# Select any of the query from above and run this. Yo can also ask another one if you want.
query = query_1

output = await full_chain.ainvoke(query)
print("------")
print(f"Trainer's question: {query}")
print(f"Pokedex: {output}")

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "Do you know metapod"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<question,pokemon_data>] Entering Chain run with input:
{
  "input": "Do you know metapod"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<question,pokemon_data> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "Do you know metapod"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<question,pokemon_data> > chain:RunnableSequence] Entering Chain run with input:
{
  "input": "Do you know metapod"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<question,pokemon_data> > chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "input": "Do you know metapod"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<question,pokemon_data> > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "Do you k